In [1]:
"""
NOTE:
Before running this, create a virtual environment running
Python 3.12
""" 
# pytorch install, cpu-only
%pip install torch 
%pip install transformers
%pip install sentence-transformers
%pip install pandas
%pip install chonkie
%pip install pypdf[image]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [17]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding

import os
import sys
import pandas as pd
import numpy as np
# Add the current directory to the path
sys.path.append(os.getcwd())
from preprocess import prepare_PDF

In [3]:
# define paths
main = r"C:\Users\Steven\Documents\Python\super-search"
data = f"{main}/data/tests"
test_file = f"{data}/32286.pdf"

In [4]:
df = pd.DataFrame.from_dict(prepare_PDF(test_file))
df

,raw_chunk,processed_chunk,file_path
0,NBER WORKING PAPER SERIES\nPROMOTING PUBLIC HE...,PROMOTING PUBLIC HEALTH WITH BLUNT INSTRUMENTS...,C:\Users\Steven\Documents\Python\super-search/...
1,"NBER publications.\n© 2024 by Rahi Abouk, John...","NBER publications. 2024 Rahi Abouk, John S. Ea...",C:\Users\Steven\Documents\Python\super-search/...
2,If mandates increase perceived safety of the ...,If mandates increase perceived safety healthca...,C:\Users\Steven\Documents\Python\super-search/...
3,mandates and are slower to be replaced than wo...,mandates slower to replaced than workers in no...,C:\Users\Steven\Documents\Python\super-search/...
4,edu 1 Introduction\nThe COVID-19 pandemic - li...,edu 1 Introduction COVID-19 pandemic - like ot...,C:\Users\Steven\Documents\Python\super-search/...
...,...,...,...
184,"Exitt Entert\nMandate states, Non-mandate Mand...","Exitt Entert Mandate states, Non-mandate Manda...",C:\Users\Steven\Documents\Python\super-search/...
185,Among HCO 0.0539 0.0492 0.0531 0.0481\n(obs: 7...,"Among HCO 0.0539 0.0492 0.0531 0.0481 (obs 7,0...",C:\Users\Steven\Documents\Python\super-search/...
186,50-59 years 0.22 0.22 0.22 0.22\n60-64 years 0...,50-59 years 0.22 0.22 0.22 0.22 60-64 years 0....,C:\Users\Steven\Documents\Python\super-search/...
187,Less than high school 0.02 0.02 0.02 0.02\nHig...,Less than high school 0.02 0.02 0.02 0.02 High...,C:\Users\Steven\Documents\Python\super-search/...


## To-Do:

### Database management
- We need a system to handle vector database as well as allowing for fast retrieval of the files corresponding to each vector
    - Each file needs a file_id
        - Links to the filepath
    - Each chunk needs a chunk_id
        - Links to the chunk text
            - ~~Importantly, link to the original text, not the processed text used for embedding.~~
                - ~~**I think the solution is to do the sentence chunking on the original text, before processing**~~
    - file_id + chunk_id should uniquely identify a vector in the database
- After taking a user query and encoding it, perform similarity search in the database
    to identify a row, then link to the row's filepath and text
- Then print a hyperlink to the file, and print the original text
- **Problem**: how to return images, given that we first caption them with an LLM and then encode the caption?

### Misc.
- Parallelized PDF processing

### Incorporating Images
- Use PyPDF to extract images from each page of the PDF.
```
for page in reader.pages:
    try:
        for image in page.images:
            extract_image(image)
    except:
        pass
```
- To render the image visually: image.image. Can also show as bytes
- Goal is to pass each image to a multi-modal LLM for summary, which is then fed into the cleaned text.
    - Possibly LLaVA for describing the images.
    - Since captioning the images would take a super long time, this should be an optional step.
        - Ideally would be done after first parsing the text, but then you might have to regenerate the whole vector base

### GUI
- The following should be customizable inputs:
    - Chunk token size ("larger is faster but less accurate") (default: 256)
    - Chunk overlap ("larger gives more context per chunk") (default: chunk_size / 4)
    - Choice of sentence transformer: provide a few options based on speed/accuracy tradeoff.
        - Fastest: static-retrieval-mrl-en-v1
        - Medium: bge-m3
        - Slowest: gte-large-en-v1.5
        - (these are subject to change)
    - Index database save location
    - Similarity matrix (default: cosine)

In [10]:
# Current encoding model implementation: static-retrieval-mrl-en-v1
# https://huggingface.co/sentence-transformers/static-retrieval-mrl-en-v1
# Model defaults to 1024 dense dimensions, but can be truncated to save space/time

truncated_dimensions = 1024

model = SentenceTransformer(
    "sentence-transformers/static-retrieval-mrl-en-v1"
    , device="cpu"
    , truncate_dim=truncated_dimensions
    )


In [13]:
vecs = model.encode(df['processed_chunk'])
# This returns a np array of shape (n, d), where n is 
#     number of chunks and d is embedding dimensions.

df2['vector'] = [i for i in np.unstack(vecs)]
# Add the embeddings to our dataframe in a single variable,
#     so each cell contains the d-dimensional np vector.

In [42]:
## TESTING
# Importing a lot of PDFs to see how long this takes
papers_repo = r"C:\Users\Steven\Documents\Python\Data\NBER papers"

files = os.listdir(papers_repo)
files.sort(reverse=True)

full_dict = {
    'raw_chunk': []
    , 'processed_chunk': []
    , 'file_path': []
}

counter=1

for paper in files[0:99]:
    f = f"{papers_repo}/{paper}"
    iter_dict = prepare_PDF(f, tokenizer_model='minishlab/potion-base-8M')
    full_dict['raw_chunk'].extend(iter_dict['raw_chunk'])
    full_dict['processed_chunk'].extend(iter_dict['processed_chunk'])
    full_dict['file_path'].extend(iter_dict['file_path'])
    counter+=1
    if counter%5==0:
        print(f"Finished filed {counter}.")

df = pd.DataFrame.from_dict(full_dict)
df

# Currently takes around 1 second per file
    

Finished filed 5.
Finished filed 10.
Finished filed 15.
Finished filed 20.
Finished filed 25.
Finished filed 30.
Finished filed 35.
Finished filed 40.
Finished filed 45.
Finished filed 50.
Finished filed 55.
Finished filed 60.
Finished filed 65.
Finished filed 70.
Finished filed 75.
Finished filed 80.
Finished filed 85.
Finished filed 90.
Finished filed 95.
Finished filed 100.


,raw_chunk,processed_chunk,file_path
0,NBER WORKING PAPER SERIES\nRETIREMENT AND THE ...,RETIREMENT AND THE EVOLUTION OF PENSION STRUCT...,C:\Users\Steven\Documents\Python\Data\NBER pap...
1,"All rights reserved. Short sections of text,...","All rights reserved. Short sections text, not ...",C:\Users\Steven\Documents\Python\Data\NBER pap...
2,"Thus, the evolution of pension structure can ...","Thus, evolution pension structure can help exp...",C:\Users\Steven\Documents\Python\Data\NBER pap...
3,DB pension wealth typically accumulates slow...,DB pension wealth typically accumulates slowly...,C:\Users\Steven\Documents\Python\Data\NBER pap...
4,DC pensions accumulate a lump sum which depe...,DC pensions accumulate lump sum which depends ...,C:\Users\Steven\Documents\Python\Data\NBER pap...
...,...,...,...
17557,786*** (.063) (.137) (.071) (.096) ...,786*** (.063) (.137) (.071) (.096) (.107) N 44...,C:\Users\Steven\Documents\Python\Data\NBER pap...
17558,Total Mixed Sex Same Sex Boys Only Gi...,Total Mixed Sex Same Sex Boys Only Girls Only ...,C:\Users\Steven\Documents\Python\Data\NBER pap...
17559,141) N 2364 698 1666 940 726 ***p...,141) N 2364 698 1666 940 726 ***p<.001 **p<.01...,C:\Users\Steven\Documents\Python\Data\NBER pap...
17560,Infant Mortality -.892*** -.703*** -.51...,Infant Mortality -.892*** -.703*** -.512*** (....,C:\Users\Steven\Documents\Python\Data\NBER pap...


In [43]:
vecs = model.encode(df['processed_chunk'])
# This returns a np array of shape (n, d), where n is 
#     number of chunks and d is embedding dimensions.

df['vector'] = [i for i in np.unstack(vecs)]
# Add the embeddings to our dataframe in a single variable,
#     so each cell contains the d-dimensional np vector.

df
# takes 3 seconds to encode 17562 chunks, 33 files per second

,raw_chunk,processed_chunk,file_path,vector
0,NBER WORKING PAPER SERIES\nRETIREMENT AND THE ...,RETIREMENT AND THE EVOLUTION OF PENSION STRUCT...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.9494798, -0.5101314, 1.260272, -3.4051301, ..."
1,"All rights reserved. Short sections of text,...","All rights reserved. Short sections text, not ...",C:\Users\Steven\Documents\Python\Data\NBER pap...,"[-0.41856015, 0.94318783, -0.1712659, -3.14227..."
2,"Thus, the evolution of pension structure can ...","Thus, evolution pension structure can help exp...",C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.24726994, 1.506088, -0.993264, -2.7431295, ..."
3,DB pension wealth typically accumulates slow...,DB pension wealth typically accumulates slowly...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[1.8708433, 0.6725198, -2.1666236, -3.2994564,..."
4,DC pensions accumulate a lump sum which depe...,DC pensions accumulate lump sum which depends ...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[1.0445604, 0.36211833, -0.17084824, -1.149713..."
...,...,...,...,...
17557,786*** (.063) (.137) (.071) (.096) ...,786*** (.063) (.137) (.071) (.096) (.107) N 44...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[-0.0041005667, -1.6556355, 2.509584, -0.22076..."
17558,Total Mixed Sex Same Sex Boys Only Gi...,Total Mixed Sex Same Sex Boys Only Girls Only ...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.13459334, -1.6275283, 2.4162931, -0.3573123..."
17559,141) N 2364 698 1666 940 726 ***p...,141) N 2364 698 1666 940 726 ***p<.001 **p<.01...,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[0.07510667, -1.8058623, 2.654449, 0.3212689, ..."
17560,Infant Mortality -.892*** -.703*** -.51...,Infant Mortality -.892*** -.703*** -.512*** (....,C:\Users\Steven\Documents\Python\Data\NBER pap...,"[-0.8045424, -2.0775964, 1.3887693, 0.8446741,..."


In [47]:
def setup_chunker(_chunk_size=256, _chunk_overlap=64):
    # Logic to adjust inputs if necessary
    if 2 * _chunk_overlap > _chunk_size:
        _chunk_overlap = round(_chunk_size / 2)
        print(f"Warning: chunk overlap too large, setting to {_chunk_overlap}.")

    def chunker(text):
        # Split text into words
        words = text.split(' ')
        chunks = []

        start = 0
        end = _chunk_size-1

        while end < len(words):
            chunk = words[start:end]
            chunks.append(chunk)

            start = end - _chunk_overlap + 1
            end = end + _chunk_overlap
        
        # make sure the last chunk has all the remaining words
        if end < len(words) - 1:
            chunks.append(words[end:])

        return(chunks)

    return(chunker)

chunker = setup_chunker(10, 5)
chunker("I think this is what I need ;) will see if it works and check answer if it does for my problem, I might still need some help, thanks everyone, this was fast :D ")

[['I', 'think', 'this', 'is', 'what', 'I', 'need', ';)', 'will'],
 ['I', 'need', ';)', 'will', 'see', 'if', 'it', 'works', 'and'],
 ['if', 'it', 'works', 'and', 'check', 'answer', 'if', 'it', 'does'],
 ['answer', 'if', 'it', 'does', 'for', 'my', 'problem,', 'I', 'might'],
 ['my', 'problem,', 'I', 'might', 'still', 'need', 'some', 'help,', 'thanks'],
 ['need', 'some', 'help,', 'thanks', 'everyone,', 'this', 'was', 'fast', ':D']]

In [45]:
chunk_size=15
chunk_overlap=5

start = 0
end = chunk_size-1

while end < 100:
    print(f"Start: {start}, End: {end}.")
    start = end - chunk_overlap + 1
    end = end + chunk_overlap

    

Start: 0, End: 14.
Start: 10, End: 19.
Start: 15, End: 24.
Start: 20, End: 29.
Start: 25, End: 34.
Start: 30, End: 39.
Start: 35, End: 44.
Start: 40, End: 49.
Start: 45, End: 54.
Start: 50, End: 59.
Start: 55, End: 64.
Start: 60, End: 69.
Start: 65, End: 74.
Start: 70, End: 79.
Start: 75, End: 84.
Start: 80, End: 89.
Start: 85, End: 94.
Start: 90, End: 99.
